In [1]:
from pdfrw import PdfReader, PdfWriter
from utils import \
    check_status, \
    create_filedir, \
    check_input_file, \
    pdf_merger, \
    convert_pdf_to_string, \
    quantity_extract, \
    read_config, \
    extract_data, \
    pdf_whitespace, \
    pdf_cropper, \
    create_count_pdf
import os
import shutil

In [2]:
check_status()
create_filedir()
all_pdf = check_input_file("input")
config = read_config()
# merge pdf and remove blank pages
print("Merging all the PDF Files")
pdf_merger(all_pdf)
print("Merge Completed")

Merging all the PDF Files
Merge Completed


In [3]:
print("Converting PDF to Text")
all_page = convert_pdf_to_string(os.path.join("temp", "output.pdf"))
print("Conversion Completed")

print("Extracting Data")
df = extract_data(all_page)
print("Extraction Completed")

Converting PDF to Text
Conversion Completed
Extracting Data


49it [00:00, 2218.68it/s]

Extraction Completed


In [4]:
df.shape

(49, 8)

In [5]:
sort_list = []
ascending_list = []
if config["sku_sort"]:
    sort_list.append("sku")
    ascending_list.append(False)

if config["courier_sort"]:
    sort_list.append("courier")
    ascending_list.append(True)
if config["soldBy_sort"]:
    sort_list.append("soldBy")
    ascending_list.append(True)

# make sku lowercase
df["sku"] = df["sku"].str.lower()
df = df.sort_values(by=sort_list, ascending=ascending_list)
df.shape

(49, 8)

In [6]:
true_rows = df[df["multi"] == True]
whole_data = df.copy(deep=True)
df = df.drop(true_rows.index)
reader_input = PdfReader("temp/output.pdf")
writer_output = PdfWriter()
for page in df.page.values:
    writer_output.addpage(reader_input.pages[page])
writer_output.write(os.path.join("temp", "output_single.pdf"))

In [7]:
single_path = pdf_whitespace(os.path.join("temp", "output_single.pdf"))
# rename whitespace pdf
os.rename(single_path, single_path.replace("_whitespace.pdf", ".pdf"))
single_path = single_path.replace("_whitespace.pdf", ".pdf")

100%|██████████| 49/49 [00:00<00:00, 49.75it/s]


In [8]:
pdf_cropper(single_path, config)

  0%|          | 0/49 [00:00<?, ?it/s]

100%|██████████| 49/49 [00:00<00:00, 327.44it/s]


In [9]:
if len(true_rows) != 0:
    writer_output = PdfWriter()
    for page in true_rows.page.values:
        writer_output.addpage(reader_input.pages[page])
    writer_output.write(os.path.join("temp", "output_multi.pdf"))
    multi_path = pdf_whitespace(os.path.join("temp", "output_multi.pdf"))
    pdf_cropper(multi_path, config)

In [10]:
if len(true_rows) == 0:
    shutil.move(os.path.join("temp", "output_single.pdf"), os.path.join("output", "result.pdf"))